In [66]:
import numpy as np
import pandas as pd

In [67]:
df=pd.read_csv(r"C:\Users\asus\OneDrive\Desktop\dataset\gurgaon_properties_post_feature_selection.csv")

In [68]:
df

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_categories,price
0,0.0,82.0,2,2,1.0,3.0,1000.0,0,0,0,1.0,2.0,0.45
1,0.0,36.0,2,2,1.0,2.0,722.0,0,0,2,1.0,1.0,0.50
2,0.0,114.0,2,2,3.0,1.0,661.0,0,0,0,1.0,0.0,0.40
3,0.0,72.0,2,2,2.0,1.0,1333.0,0,0,0,2.0,1.0,1.47
4,0.0,108.0,2,2,3.0,4.0,1217.0,0,0,0,1.0,2.0,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,1.0,67.0,3,3,3.0,0.0,1650.0,0,1,2,2.0,1.0,4.75
3487,1.0,33.0,4,4,3.0,0.0,1800.0,1,0,0,2.0,1.0,5.50
3488,1.0,32.0,3,2,3.0,2.0,1350.0,0,0,0,1.0,1.0,4.25
3489,1.0,33.0,3,3,2.0,0.0,1350.0,1,0,0,2.0,1.0,4.50


In [69]:
#one hot encode -> sector,balcony ,agePosssession, funishing type ,luxury category , floor category
y=df['price']
x=df.drop(columns=['price'])

In [70]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [71]:
columns_to_encode=['sector','balcony','agePossession','furnishing_type','luxury_category','floor_categories']

In [72]:
y_transformed=np.log1p(y)

In [73]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), columns_to_encode)
    ],
    remainder='passthrough'
)


In [83]:
pipeline=Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',SVR()) 
])

In [84]:
kfold=KFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_val_score(pipeline,x,y_transformed,cv=kfold,scoring='r2')

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [85]:
scores.mean()

0.892599952838801

In [86]:
scores.std()

0.014532728688802236

In [87]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y_transformed,test_size=0.2,random_state=42)

In [88]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_categories'])])),
                ('regressor', SVR())])

In [89]:
y_pred=pipeline.predict(x_test)

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [90]:
y_pred=np.expm1(y_pred)

In [91]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5923344108359774